In [1]:
import numpy as np
import pandas as pd
#from sklearn.neighbors import NearestNeighbors as NN


In [2]:
#useful constants:
infty = 1e100
neigh_number = 1 #will probably be changed during testing

In [3]:
#metrics

#general template:
#arguments: pair of vectors of scores; these vectors should be dictionaries in form {anime_id : rating}

#returns: distance between given vectors

def normalized_dist(x, y) :
    nonzero_indices = x.keys() & y.keys()
    N = len(nonzero_indices)
    
    if(N == 0) :
        return infty #defined at the beginning, check for that
    
    cut_x = np.array([x[ind] for ind in nonzero_indices])
    cut_y = np.array([y[ind] for ind in nonzero_indices])
    diff = cut_x - cut_y
    
    dist = np.sum(diff ** 2) / N
    return dist



In [4]:
#arguments: 
# x - main vector, vectors - all other vectors
# dist - metric used (function), K - number of neighbors used to calculate the score

#returns list of K (or less if there are less in total) neighbors
def myKNN(x, vectors, dist, K) :
    dists = []
    ret = []
    
    N = len(vectors)
    for i in range(0, N) :
        dists.append((dist(x, vectors[i]), i))
    
    dists = sorted(dists)
    
    K = min(K, N)
    for i in range(0, K) :
        ret.append(dists[i][1])
    
    return ret
    

In [5]:
#arguments: 
# anime_id - index of user, user_vec - user's ratings as vector, vectors - other users' ratings as vectors
# dist - metric used (function), K - number of neighbors used to calculate the score

# returns score based on nearest neighbours
def KNN_score(anime_id, user_vec, vectors, dist, K = 5) :
    neighbors = myKNN(user_vec, vectors, dist, K)
    neigh_scores = np.array([vectors[nei][anime_id] for nei in neighbors])
    
    score = np.average(neigh_scores)
    return score
    

In [6]:
#read the data
records = pd.read_csv('rating.csv')
records = records.to_numpy()

users = 0
animes = 0

for r in records :
    users = max(users, r[0])
    animes = max(animes, r[1])

print("users:", users, "animes:", animes)


users: 73516 animes: 34519


In [7]:
ratings = [0] * (users + 1)
count_anime = [0] * (users + 1)
count_users = [0] * (animes + 1)

for i in range(0, users + 1) :
    ratings[i] = dict()

#optional: minus_one - new value for all -1s (0 means ignoring them)
def parse_records(minus_one = -1) :
    for r in records :
        user_id, anime_id, rating = r
        if(rating == -1) :
            rating = minus_one
        if(rating != 0) : 
            ratings[user_id][anime_id] = rating
            if(rating != -1) :
                count_users[anime_id] += 1
                count_anime[user_id] += 1
                

#argument: anime_id

#returns: list of rating vectors of users who watched anime_id
def cut_records(anime_id) :
    result = []
    
    for user in range(1, users + 1) :
        if(anime_id in ratings[user]) :
            result.append(ratings[user])
    
    return result


parse_records()
    

In [8]:
#arguments: user_id, anime_id
#optional: if score should be rounded, if it is for testing purpouse

#returns estimation of the score
def estimate_score(user_id, anime_id, rounded = True, test = False) :    
    temp = 0

    if(test == True) :
        temp = ratings[user_id].pop(anime_id, None)
        if(temp == None) :
            print("You sholud test on existing records! (try test = False)")
            return 2137.0
    else :
        if(anime_id in ratings[user_id]) :
            print("Anime already rated (forgot test = False ?)")
            return ratings[user_id][anime_id]
    
    watched = cut_records(anime_id)
            
    answer = KNN_score(anime_id, ratings[user_id], watched, normalized_dist, K = neigh_number)
    
    if(test == True) :
        ratings[user_id][anime_id] = temp
    
    if(rounded == True) :
        answer = np.rint(answer)
    return answer

In [9]:
#arguments: two np.array with equal lengths: estimated and expected ratings 

#returns: average of squared differences
def error_avsq(estimated, real) :
    return np.sum((estimated-real)**2) / len(real)


In [14]:
#optional: user_bound, anime_bound, stop_bound

#returns at most stop_bound pairs (user_id, anime_id) such that:
# - used_id watched anime_id and rated it
# - user_id watched at least user_bound animes
# - anime id has been watched at least anime_bound times
def select_popular_probe(user_bound = 10, anime_bound = 10000, stop_bound = 10) :
    ans = []
    
    perm = np.arange(len(records))
    np.random.shuffle(perm)
    
    for p in perm :
        user_id, anime_id, rating = records[p]
        if(rating > 0 and count_anime[user_id] >= user_bound and count_users[anime_id] >= anime_bound) :
            ans.append((user_id, anime_id))
            if(len(ans) == stop_bound) :
                break
                
    return ans

def select_random_probe(stop_bound = 10) :
    return select_popular_probe(0, 0, stop_bound)


In [16]:
#arguments: k - parameter for k nearest neighbors algorithm,
# test_pairs - testset in form of pairs (user_id, anime_id)
#optional: function measuring errors

#prints errors
def make_test(k, test_pairs, error_func = error_avsq) :
    global neigh_number
    neigh_number = k
    
    N = len(test_pairs)
    
    estimated = []
    not_rounded = []
    expected = []
    
    for t in test_pairs:
        user_id, anime_id = t
        print(user_id, anime_id)
        estimated.append(estimate_score(user_id, anime_id, test = True))
        not_rounded.append(estimate_score(user_id, anime_id, test = True, rounded = False))
        expected.append(ratings[user_id][anime_id])
    
    estimated = np.array(estimated)
    expected = np.array(expected)
    not_rounded = np.array(not_rounded)

    
    error = error_func(estimated, expected)
    error_not_rounded = error_func(not_rounded, expected)
    
    print("k =", k, "error =", error, "error without rounding =", error_not_rounded)
    

In [17]:
test_pairs = select_popular_probe()
for k in range(1, 21) :
    make_test(k, test_pairs)
    
test_pairs = select_random_probe()
for k in range(1, 21) :
    make_test(k, test_pairs)
    

7520 889
38523 5114
62135 11617
37465 4181
21666 2993
50287 6547
23081 431
14573 20
62782 13601
61318 5680
k = 1 error = 13.2 error without rounding = 13.2
7520 889
38523 5114
62135 11617
37465 4181
21666 2993
50287 6547
23081 431
14573 20
62782 13601
61318 5680
k = 2 error = 7.9 error without rounding = 7.175
7520 889
38523 5114
62135 11617
37465 4181
21666 2993
50287 6547
23081 431
14573 20
62782 13601
61318 5680
k = 3 error = 4.6 error without rounding = 4.722222222222223
7520 889
38523 5114
62135 11617
37465 4181
21666 2993
50287 6547
23081 431
14573 20
62782 13601
61318 5680
k = 4 error = 3.0 error without rounding = 2.85625
7520 889
38523 5114
62135 11617
37465 4181
21666 2993
50287 6547
23081 431
14573 20
62782 13601
61318 5680
k = 5 error = 2.2 error without rounding = 2.316
7520 889
38523 5114
62135 11617
37465 4181
21666 2993
50287 6547
23081 431
14573 20
62782 13601
61318 5680
k = 6 error = 1.7 error without rounding = 1.6944444444444446
7520 889
38523 5114
62135 11617
37465